In [1]:
# we must import the libraries once again since we haven't imported them in this file
import numpy as np
import tensorflow as tf

### Data

In [2]:
npz = np.load('Podcast_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

# we load the validation data in the temporary variable
npz = np.load('Podcast_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load('Podcast_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model
Outline, optimizers, loss, early stopping and training

In [3]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 0.6043 - accuracy: 0.6700 - val_loss: 0.5242 - val_accuracy: 0.7002
Epoch 2/100
3579/3579 - 0s - loss: 0.4569 - accuracy: 0.7720 - val_loss: 0.4601 - val_accuracy: 0.7472
Epoch 3/100
3579/3579 - 0s - loss: 0.4099 - accuracy: 0.7829 - val_loss: 0.4237 - val_accuracy: 0.7673
Epoch 4/100
3579/3579 - 0s - loss: 0.3869 - accuracy: 0.7969 - val_loss: 0.4112 - val_accuracy: 0.7673
Epoch 5/100
3579/3579 - 0s - loss: 0.3711 - accuracy: 0.8033 - val_loss: 0.3960 - val_accuracy: 0.7785
Epoch 6/100
3579/3579 - 0s - loss: 0.3623 - accuracy: 0.8016 - val_loss: 0.3847 - val_accuracy: 0.7875
Epoch 7/100
3579/3579 - 0s - loss: 0.3536 - accuracy: 0.8111 - val_loss: 0.3803 - val_accuracy: 0.7718
Epoch 8/100
3579/3579 - 0s - loss: 0.3507 - accuracy: 0.8069 - val_loss: 0.3903 - val_accuracy: 0.7584
Epoch 9/100
3579/3579 - 0s - loss: 0.3439 - accuracy: 0.8125 - val_loss: 0.3647 - val_accuracy: 0.7830
Epoch 10/100
3579/3579 - 0

## Test the model

As we discussed in the lectures, after training on the training data and validating on the validation data, we test the final prediction power of our model by running it on the test dataset that the algorithm has NEVER seen before.

It is very important to realize that fiddling with the hyperparameters overfits the validation dataset. 

The test is the absolute final instance. You should not test before you are completely done with adjusting your model.

If you adjust your model after testing, you will start overfitting the test dataset, which will defeat its purpose.

In [4]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [5]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.35. Test accuracy: 79.69%
